In [2]:
#last update march 6 8pm
import os
import numpy as np
import random
import math
import json

from functools import partial

## Imports for plotting
import matplotlib.pyplot as plt
plt.set_cmap('cividis')
#%matplotlib inline
#from IPython.display import set_matplotlib_formats
#set_matplotlib_formats('svg', 'pdf') # For export
from matplotlib.colors import to_rgb
import matplotlib
matplotlib.rcParams['lines.linewidth'] = 2.0
import seaborn as sns
sns.reset_orig()

## tqdm for loading bars
from tqdm.notebook import tqdm

## PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
import torch.optim as optim
from torch.types import _size

## Torchvision
import torchvision
from torchvision.datasets import CIFAR100
from torchvision import transforms


## Audio
from transformers import AutoProcessor, ASTModel
from datasets import load_dataset
from transformers import AutoProcessor, ASTModel

####image
from PIL import Image
import requests
from transformers import AutoProcessor, CLIPVisionModel , CLIPVisionModelWithProjection
torch.cuda.empty_cache()
import gc
gc.collect()
from transformers import AutoTokenizer, CLIPTextModel,AutoModel


<Figure size 640x480 with 0 Axes>

In [3]:
from model import Transformer
#from dataset import BilingualDataset, causal_mask
from config import get_config, get_weights_file_path, latest_weights_file_path

#import torchtext.datasets as datasets
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
from torch.optim.lr_scheduler import LambdaLR

import warnings
from tqdm import tqdm
import os
from pathlib import Path

# Huggingface datasets and tokenizers
from datasets import load_dataset
from tokenizers import Tokenizer
from tokenizers.models import WordLevel
from tokenizers.trainers import WordLevelTrainer
from tokenizers.pre_tokenizers import Whitespace

import torchmetrics

from torch.utils.tensorboard import SummaryWriter

from datasets import Audio
from datasets import Dataset, Image
from numpy.linalg import norm

<Figure size 640x480 with 0 Axes>

In [4]:
from datasets import load_dataset
dataset = load_dataset("detection-datasets/coco")

Found cached dataset parquet (/home/maithri/.cache/huggingface/datasets/detection-datasets___parquet/detection-datasets--coco-64ef6d5414f6b8df/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7)


  0%|          | 0/2 [00:00<?, ?it/s]

/home/maithri/.local/lib/python3.10/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)


In [7]:
#dataset_test=load_dataset('detection-datasets/coco', split='train').train_test_split(train_size=117200)
#dataset_test=load_dataset('detection-datasets/coco', split='train').train_test_split(train_size=10000)
#dataset_val=load_dataset('detection-datasets/coco', split='val')

In [9]:
import config
# config the database  
def get_ds():
    #dataset_test=load_dataset('detection-datasets/coco', split='train').train_test_split(train_size=117200)
    dataset_test=load_dataset('detection-datasets/coco', split='train').train_test_split(train_size=50000)
    dataset_val=load_dataset('detection-datasets/coco', split='val') 
    
   # train_ds_size=int(0.9*len(dataset))
    #val_ds_size= len(dataset) - train_ds_size
    #train_dataset_both ,val_dataset_both = random_split(dataset ,[train_ds_size, val_ds_size])
    #batch_size=config['batch_size'] 
    train_dataloader = 1
    val_dataloader = 1
    
    return dataset_test['train'] ,dataset_val ,train_dataloader,val_dataloader

In [12]:
def cocolabel():
    
    cococlasslabel=["person",
        "bicycle",
        "car",
        "motorcycle",
        "airplane",
        "bus",
        "train",
        "truck",
        "boat",
        "traffic light",
        "fire hydrant",
        "stop sign",
        "parking meter",
        "bench",
        "bird",
        "cat",
        "dog",
        "horse",
        "sheep",
        "cow",
        "elephant",
        "bear",
        "zebra",
        "giraffe",
        "backpack",
        "umbrella",
        "handbag",
        "tie",
        "suitcase",
        "frisbee",
        "skis",
        "snowboard",
        "sports ball",
        "kite",
        "baseball bat",
        "baseball glove",
        "skateboard",
        "surfboard",
        "tennis racket",
        "bottle",
        "wine glass",
        "cup",
        "fork",
        "knife",
        "spoon",
        "bowl",
        "banana",
        "apple",
        "sandwich",
        "orange",
        "broccoli",
        "carrot",
        "hot dog",
        "pizza",
        "donut",
        "cake",
        "chair",
        "couch",
        "potted plant",
        "bed",
        "dining table",
        "toilet",
        "tv",
        "laptop",
        "mouse",
        "remote",
        "keyboard",
        "cell phone",
        "microwave",
        "oven",
        "toaster",
        "sink",
        "refrigerator",
        "book",
        "clock",
        "vase",
        "scissors",
        "teddy bear",
        "hair drier",
        "toothbrush"]
   
    return cococlasslabel

In [13]:
import config
def get_model(): 
    model = Transformer(config["d_model"], config["ffn_hidden"], config["num_heads"], config["drop_prob"], config["num_layers"], config["kn_vocab_size"])
    return model

In [14]:
def get_image_model():

    image_model = CLIPVisionModel.from_pretrained("openai/clip-vit-base-patch32",output_hidden_states =True,output_attentions=True)
    image_processor = AutoProcessor.from_pretrained("openai/clip-vit-base-patch32")

    return image_model,image_processor 

In [15]:
def get_audio_model():

    processor_audio = AutoProcessor.from_pretrained("MIT/ast-finetuned-audioset-10-10-0.4593")
    model_audio = ASTModel.from_pretrained("MIT/ast-finetuned-audioset-10-10-0.4593", output_hidden_states =True,output_attentions=True)
    

    return model_audio,processor_audio  

In [16]:
def get_text_model():
    text_model = AutoModel.from_pretrained("sentence-transformers/all-mpnet-base-v2",output_hidden_states =True,output_attentions=True)
    text_Tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-mpnet-base-v2")
    return text_model ,text_Tokenizer 

In [17]:
def get_hidden_layer():
    
    image_layerno_1=10
    image_layerno_2=9
    print("image layer number",image_layerno_1,image_layerno_2)
    return image_layerno_1 ,image_layerno_2

In [18]:

def run_validation(model, validation_ds, device, print_msg, global_step, writer, num_examples=10):
    model.eval()
    count = 0
    image_model,image_processor =get_image_model()
    model_audio,processor_audio =get_audio_model()
    text_model,text_Tokenizer =get_text_model()

    source_image = []
    expected = []
    predicted = []
    realclass_count=0
    number_inputs=0
    try:
        # get the console window width
        with os.popen('stty size', 'r') as console:
            _, console_width = console.read().split()
            console_width = int(console_width)
    except:
        # If we can't get the console width, use 80 as default
        console_width = 80

    preds = []

    image_layerno_1,image_layerno_2= get_hidden_layer()
  
    #batch_size=config['batch_size'] 
    batch_size=4
    batch_iterator = tqdm(validation_ds.iter(batch_size))

    with torch.no_grad():
        for batch in batch_iterator :
            count += 1

########################  image KQV extraction 
            cropped_image=[]
            allclass_numers=[]  
    
########################  image KQV extraction 
            num_image=0
            images_list=batch['image']
            #print(images_list)
            allbbox=batch['objects']
            #allclass_num=batch['objects']['category']
            #print('numberofclass',allclass_num)
########################  image KQV extraction 
            for bb in range(0,batch_size ):    
                num_objects=len(allbbox[bb]['bbox'])
                #allclass_numers.append(batch['objects'][bb]['category'])
                #print('numberofclass',allclass_numers)
                #print("numberof object at image ",num_objects)
                images=images_list[bb].convert("RGB")
                for a in range(0,num_objects) :
                  #  print(allbbox[bb]['bbox'][a])
                    cropped_image.append(images.crop(allbbox[bb]['bbox'][a]) ) 
                    allclass_numers.append(batch['objects'][bb]['category'][a])
                    num_image=num_image+1

            image_1 = image_processor(images=cropped_image, return_tensors="pt").to(device)
            image_model=image_model.to(device)
            image_outputs_1 = image_model(**image_1)
            image_pooled_output_1 = image_outputs_1.pooler_output  
            image_layear_output_1=image_outputs_1.hidden_states[image_layerno_1],image_outputs_1.hidden_states[image_layerno_2]


                  

            coco_classes = cocolabel()
            rlabel = [None] * num_image
            for x in range(num_image) :
                rlabel[x]=coco_classes[allclass_numers[x]]
            

           # cifar10_classes = ['airplane']
            
          #  rlabel = [None] * batch_size
          
         #   for x in range(batch_size) :
          #      rlabel[x]=cifar10_classes[label[x]]


            inputs_text = text_Tokenizer(text=coco_classes, padding=True, return_tensors="pt").to(device)
            text_model= text_model.to(device)
            text_outputs = text_model(**inputs_text)
            text_pooled_output = text_outputs.pooler_output
           # print("text eddddddddddddddddddddddd",text_pooled_output.shape )
            #text_layear_output=text_outputs.hidden_states[image_layerno_1],text_outputs.hidden_states[image_layerno_2]

###########################################################################

  
#############################################################################


            #Text model parameters

  #          text_query_bias=text_model.encoder.layer[10].attention.self.query.bias,text_model.encoder.layer[9].attention.self.query.bias 
            
 #           text_key_bias=text_model.encoder.layer[10].attention.self.key.bias,text_model.encoder.layer[9].attention.self.key.bias
 ###           text_value_bias=text_model.encoder.layer[10].attention.self.value.bias,text_model.encoder.layer[9].attention.self.value.bias
 #           _start = 0
  #          _end = config["d_model"]
  #          text_query_weight=text_model.encoder.layer[10].attention.self.query.weight[_start:_end, :],text_model.encoder.layer[9].attention.self.query.weight[_start:_end, :]
#            text_key_weight=text_model.encoder.layer[10].attention.self.key.weight[_start:_end, :],text_model.encoder.layer[9].attention.self.key.weight[_start:_end, :]
 #           text_value_weight=text_model.encoder.layer[10].attention.self.value.weight[_start:_end, :],text_model.encoder.layer[9].attention.self.value.weight[_start:_end, :]


            text_query_bias=text_model.encoder.layer[image_layerno_1].attention.attn.q.bias,text_model.encoder.layer[image_layerno_2].attention.attn.q.bias 
            text_key_bias=text_model.encoder.layer[image_layerno_1].attention.attn.k.bias,text_model.encoder.layer[image_layerno_2].attention.attn.k.bias
            text_value_bias=text_model.encoder.layer[image_layerno_1].attention.attn.v.bias,text_model.encoder.layer[image_layerno_2].attention.attn.v.bias
            _start = 0
            _end = config["d_model"]
            text_query_weight=text_model.encoder.layer[image_layerno_1].attention.attn.q.weight[_start:_end, :],text_model.encoder.layer[image_layerno_2].attention.attn.q.weight[_start:_end, :]
            text_key_weight=text_model.encoder.layer[image_layerno_1].attention.attn.k.weight[_start:_end, :],text_model.encoder.layer[image_layerno_2].attention.attn.k.weight[_start:_end, :]
            text_value_weight=text_model.encoder.layer[image_layerno_1].attention.attn.v.weight[_start:_end, :],text_model.encoder.layer[image_layerno_2].attention.attn.v.weight[_start:_end, :]



            #image model parameters 
            image_query_bias=image_model.vision_model.encoder.layers[image_layerno_1].self_attn.q_proj.bias,image_model.vision_model.encoder.layers[image_layerno_2].self_attn.q_proj.bias
            image_key_bias=image_model.vision_model.encoder.layers[image_layerno_1].self_attn.k_proj.bias,image_model.vision_model.encoder.layers[image_layerno_2].self_attn.k_proj.bias
            image_value_bias=image_model.vision_model.encoder.layers[image_layerno_1].self_attn.v_proj.bias,image_model.vision_model.encoder.layers[image_layerno_2].self_attn.v_proj.bias

            image_query_weight=image_model.vision_model.encoder.layers[image_layerno_1].self_attn.q_proj.weight[_start:_end, :],image_model.vision_model.encoder.layers[image_layerno_2].self_attn.q_proj.weight[_start:_end, :]
            image_key_weight=image_model.vision_model.encoder.layers[image_layerno_1].self_attn.k_proj.weight[_start:_end, :],image_model.vision_model.encoder.layers[image_layerno_2].self_attn.k_proj.weight[_start:_end, :]
            image_value_weight=image_model.vision_model.encoder.layers[image_layerno_1].self_attn.v_proj.weight[_start:_end, :],image_model.vision_model.encoder.layers[image_layerno_2].self_attn.v_proj.weight[_start:_end, :]

            #proj_output=model(config['d_model'], config['ffn_hidden'], config['num_heads'], config['drop_prob'], config['num_layers'], config['kn_vocab_size'], model_audio, image_model, outputs_audio, image_outputs)
            appfor= []
            scores_1 = [None] * 80
            scores_2 = [None] * 80
            scores_3 = [None] * 80
            scores_4 = [None] * 80
            
            for img in range(0,num_image):
              image_1 = image_processor(images=cropped_image[img], return_tensors="pt").to(device)
              image_model=image_model.to(device)
              image_outputs_1 = image_model(**image_1)
              image_pooled_output_1 = image_outputs_1.pooler_output  
              image_layear_output_1=image_outputs_1.hidden_states[image_layerno_1],image_outputs_1.hidden_states[image_layerno_2]
              """
              image_2 = image_processor(images=cropped_image[img+1], return_tensors="pt").to(device)
              image_model=image_model.to(device)
              image_outputs_2 = image_model(**image_2)
              image_pooled_output_2 = image_outputs_2.pooler_output  
              image_layear_output_2=image_outputs_2.hidden_states[image_layerno_1],image_outputs_2.hidden_states[image_layerno_2]
              
              
              image_3 = image_processor(images=cropped_image[img+2], return_tensors="pt").to(device)
              image_model=image_model.to(device)
              image_outputs_3 = image_model(**image_3)
              image_pooled_output_3 = image_outputs_3.pooler_output  
              image_layear_output_3=image_outputs_3.hidden_states[image_layerno_1],image_outputs_3.hidden_states[image_layerno_2]
              
              image_4 = image_processor(images=cropped_image[img+3], return_tensors="pt").to(device)
              image_model=image_model.to(device)
              image_outputs_4 = image_model(**image_4)
              image_pooled_output_4 = image_outputs_4.pooler_output  
              image_layear_output_4=image_outputs_4.hidden_states[image_layerno_1],image_outputs_4.hidden_states[image_layerno_2]
              """
              
              #image model parameters 
              image_query_bias=image_model.vision_model.encoder.layers[image_layerno_1].self_attn.q_proj.bias,image_model.vision_model.encoder.layers[image_layerno_2].self_attn.q_proj.bias
              image_key_bias=image_model.vision_model.encoder.layers[image_layerno_1].self_attn.k_proj.bias,image_model.vision_model.encoder.layers[image_layerno_2].self_attn.k_proj.bias
              image_value_bias=image_model.vision_model.encoder.layers[image_layerno_1].self_attn.v_proj.bias,image_model.vision_model.encoder.layers[image_layerno_2].self_attn.v_proj.bias

              image_query_weight=image_model.vision_model.encoder.layers[image_layerno_1].self_attn.q_proj.weight[_start:_end, :],image_model.vision_model.encoder.layers[image_layerno_2].self_attn.q_proj.weight[_start:_end, :]
              image_key_weight=image_model.vision_model.encoder.layers[image_layerno_1].self_attn.k_proj.weight[_start:_end, :],image_model.vision_model.encoder.layers[image_layerno_2].self_attn.k_proj.weight[_start:_end, :]
              image_value_weight=image_model.vision_model.encoder.layers[image_layerno_1].self_attn.v_proj.weight[_start:_end, :],image_model.vision_model.encoder.layers[image_layerno_2].self_attn.v_proj.weight[_start:_end, :]

              
              scores = [None] * 80
              for x in range(0, 80):
  
                inputs_text = text_Tokenizer(text=coco_classes[x], padding=True, return_tensors="pt").to(device)
                text_model= text_model.to(device)
                text_outputs = text_model(**inputs_text)
                text_pooled_output = text_outputs.pooler_output
                
                
                
                text_query_bias=text_model.encoder.layer[image_layerno_1].attention.attn.q.bias,text_model.encoder.layer[image_layerno_2].attention.attn.q.bias 
                text_key_bias=text_model.encoder.layer[image_layerno_1].attention.attn.k.bias,text_model.encoder.layer[image_layerno_2].attention.attn.k.bias
                text_value_bias=text_model.encoder.layer[image_layerno_1].attention.attn.v.bias,text_model.encoder.layer[image_layerno_2].attention.attn.v.bias
                _start = 0
                _end = config["d_model"]
                text_query_weight=text_model.encoder.layer[image_layerno_1].attention.attn.q.weight[_start:_end, :],text_model.encoder.layer[image_layerno_2].attention.attn.q.weight[_start:_end, :]
                text_key_weight=text_model.encoder.layer[image_layerno_1].attention.attn.k.weight[_start:_end, :],text_model.encoder.layer[image_layerno_2].attention.attn.k.weight[_start:_end, :]
                text_value_weight=text_model.encoder.layer[image_layerno_1].attention.attn.v.weight[_start:_end, :],text_model.encoder.layer[image_layerno_2].attention.attn.v.weight[_start:_end, :]


                text_layear_output=text_outputs.hidden_states[image_layerno_1],text_outputs.hidden_states[image_layerno_2]
                text_pooled_output =text_pooled_output.detach().cpu().numpy()
                  
                batch_size_new=1  
                proj_output_1=model(batch_size_new,text_query_bias,text_key_bias,text_value_bias,text_query_weight,text_key_weight,text_value_weight,image_query_bias, image_key_bias,image_value_bias,
                                      image_query_weight,image_key_weight,image_value_weight, text_layear_output, image_layear_output_1, image_pooled_output_1).to(device)
                proj_output_1 = proj_output_1.detach().cpu().numpy()
                """
                proj_output_2=model(batch_size_new,text_query_bias,text_key_bias,text_value_bias,text_query_weight,text_key_weight,text_value_weight,image_query_bias, image_key_bias,image_value_bias,
                                    image_query_weight,image_key_weight,image_value_weight, text_layear_output, image_layear_output_2, image_pooled_output_2).to(device)  
                      
                proj_output_2 = proj_output_2.detach().cpu().numpy()

              
                
                proj_output_3=model(batch_size_new,text_query_bias,text_key_bias,text_value_bias,text_query_weight,text_key_weight,text_value_weight,image_query_bias, image_key_bias,image_value_bias,
                                    image_query_weight,image_key_weight,image_value_weight, text_layear_output, image_layear_output_3, image_pooled_output_3).to(device)
                    
                proj_output_3 = proj_output_3.detach().cpu().numpy()
   
                  

                    
                proj_output_4=model(batch_size_new,text_query_bias,text_key_bias,text_value_bias,text_query_weight,text_key_weight,text_value_weight,image_query_bias, image_key_bias,image_value_bias,
                                    image_query_weight,image_key_weight,image_value_weight, text_layear_output, image_layear_output_4, image_pooled_output_4).to(device)  
                          
                proj_output_4 = proj_output_4.detach().cpu().numpy()   
                    

                """

 
                scores_1[x] = np.dot(proj_output_1, text_pooled_output.T)/(norm(proj_output_1)*(norm(text_pooled_output))) 
                #scores_2[x] = np.dot(proj_output_2, text_pooled_output.T)/(norm(proj_output_2)*(norm(text_pooled_output))) 
                #scores_3[x] = np.dot(proj_output_3, text_pooled_output.T)/(norm(proj_output_3)*(norm(text_pooled_output))) 
                #scores_4[x] = np.dot(proj_output_4, text_pooled_output.T)/(norm(proj_output_4)*(norm(text_pooled_output))) 

            
              real_class=np.argmax(scores_1)
           
             # print(allclass_numers[img],real_class)
               
              #for a in range(0, 4): 
              #  print(allclass_numers[img+a],real_class)
               # if allclass_numers[img+a] == real_class[a]:
               #    realclass_count=realclass_count+1
         
                #  print("wwwwwwwwwww",number_inputs)
              #number_inputs=number_inputs+4
               
              #print(allclass_numers[img],real_class)
              if allclass_numers[img] == real_class:
                  realclass_count=realclass_count+1
         
               #   print("wwwwwwwwwww")
              number_inputs=number_inputs+1
          
            #print("fsfdsfds",realclass_count,number_inputs)
               
            #print("cccccccccccccccccccc",np.argmax(scores))
            #scores2=np.argmax(appfor, axis=1)

            #print("scores eddddddddddddddddddddddd",scores2)
            #print("scores eddddddddddddddddddddddd",np.argmax(scores, axis=1))
           # print("compare two class",label)

      

            

            #source_image.append(scores)
            expected.append(1.00)
            predicted.append(scores)
            print("Accuracy",realclass_count,number_inputs)
            

   # true_preds = []
   # for i, label in enumerate(validation_ds["label"]):
    #    if label == preds[i]:
   #        true_preds.append(1)
   #     else:
     #      true_preds.append(0)

    Accuracy =(realclass_count / number_inputs*100 )

    print("Accuracy", Accuracy,realclass_count,number_inputs, "%")
    return Accuracy

In [19]:
import cv2


In [20]:

def train_model(config):
    device = "cuda" if torch.cuda.is_available() else "mps" if torch.has_mps or torch.backends.mps.is_available() else "cpu"
   #device="cpu"
    print("Using device:", device)
    torch.autograd.set_detect_anomaly(True)
    # Make sure the weights folder exists
    Path(f"{config['model_folder']}").mkdir(parents=True, exist_ok=True)
    
    train_dataset_both ,val_dataset_both ,train_dataloader,val_dataloader= get_ds()
   # train_dataset_both2 =train_dataset_both.shard(num_shards=10, index=0)
    print("Size of the dataset :", len(train_dataset_both))
    image_model,image_processor =get_image_model()
    model_audio,processor_audio =get_audio_model()
    text_model,text_Tokenizer =get_text_model()
    image_layerno_1,image_layerno_2= get_hidden_layer()
    

    model = get_model().to(device)

     # Tensorboard
    writer = SummaryWriter(config['experiment_name'])


   # optimizer = torch.optim.Adam(model.parameters(), lr=config['lr'], eps=1e-9)
    optimizer = torch.optim.Adam(model.parameters(), lr=config['lr'], eps=1e-9)
    value_opt = torch.optim.Adam(params = model.parameters(), lr = config['lr'])

    # If the user specified a model to preload before training, load it
    initial_epoch = 0
    global_step = 0
    preload = config['preload']
    model_filename = latest_weights_file_path(config) if preload == 'latest' else get_weights_file_path(config, preload) if preload else None
    if model_filename:
        print(f'Preloading model {model_filename}')
        state = torch.load(model_filename)
        model.load_state_dict(state['model_state_dict'])
        initial_epoch = state['epoch'] + 1
        optimizer.load_state_dict(state['optimizer_state_dict'])
        global_step = state['global_step']
    else:
        print('No model to preload, starting from scratch')

    #loss_fn = nn.CrossEntropyLoss(label_smoothing=0.1).to(device)
    loss_fn = nn.CrossEntropyLoss(label_smoothing=0.1).to(device)   
    

    
    #siprint("image layer number",image_layerno_1)

   # abc=torch.rand(1, 768).to(device)
    print("Layers ",config["num_layers"])
    for epoch in range(initial_epoch, config['num_epochs']):
        #print("epoch",epoch)
        
        torch.cuda.empty_cache()
        #train_dataset_both ,val_dataset_both ,train_dataloader,val_dataloader= get_ds()
        model.train()
        batch_size=config['batch_size'] 
        
        batch_iterator = tqdm(train_dataset_both.iter(batch_size), desc=f"Processing Epoch {epoch:02d}")
        #print("batch_iterator,batch_iterator",batch_iterator)
        

        
        for batch in batch_iterator:
            cropped_image=[]
            allclass_numers=[]  
    
########################  image KQV extraction 
            num_image=0
            images_list=batch['image']
            #print(images_list)
            allbbox=batch['objects']
            #allclass_num=batch['objects']['category']
            #print('numberofclass',allclass_num)
########################  image KQV extraction 
            for bb in range(0,batch_size ):    
                num_objects=len(allbbox[bb]['bbox'])
                #allclass_numers.append(batch['objects'][bb]['category'])
                #print('numberofclass',allclass_numers)
                #print("numberof object at image ",num_objects)
                images=images_list[bb].convert("RGB")
                for a in range(0,num_objects) :
                  #  print(allbbox[bb]['bbox'][a])
                    cropped_image.append(images.crop(allbbox[bb]['bbox'][a]) ) 
                    allclass_numers.append(batch['objects'][bb]['category'][a])
                    num_image=num_image+1
            
            #print("batch_images",num_image)         
                
            #print("images",cropped_image)    
                
            #allclass_numers2=np.squeeze(allclass_numers)
            #print('numberofclass',allclass_numers )
            
            image = image_processor(images=cropped_image, return_tensors="pt").to(device)


            image_model=image_model.to(device)
            image_outputs = image_model(**image)
            image_last_hidden_state = image_outputs.last_hidden_state
            image_pooled_output = image_outputs.pooler_output  
           # batch_nummber+= 1
         
            image_layear_output=image_outputs.hidden_states[image_layerno_1],image_outputs.hidden_states[image_layerno_2]

            

            coco_classes = cocolabel()
            rlabel = [None] * num_image
            for x in range(num_image) :
                rlabel[x]=coco_classes[allclass_numers[x]]

           
            #print("imageddddddddddddddddddddddd",label,flabel)
            inputs_text = text_Tokenizer(text=rlabel, padding=True, return_tensors="pt").to(device)
            
            text_model= text_model.to(device)
            text_outputs = text_model(**inputs_text)
            text_pooled_output = text_outputs.pooler_output

            text_layear_output=text_outputs.hidden_states[image_layerno_1],text_outputs.hidden_states[image_layerno_2]
            
            batch_size_new=len(allclass_numers)
            #audio model parameters
          #  text_query_bias=text_model.encoder.layer[10].attention.self.query.bias,text_model.encoder.layer[9].attention.self.query.bias 
         #   text_key_bias=text_model.encoder.layer[10].attention.self.key.bias,text_model.encoder.layer[9].attention.self.key.bias
          #  text_value_bias=text_model.encoder.layer[10].attention.self.value.bias,text_model.encoder.layer[9].attention.self.value.bias
          #  _start = 0
          #  _end = config["d_model"]
           # text_query_weight=text_model.encoder.layer[10].attention.self.query.weight[_start:_end, :],text_model.encoder.layer[9].attention.self.query.weight[_start:_end, :]
           # text_key_weight=text_model.encoder.layer[10].attention.self.key.weight[_start:_end, :],text_model.encoder.layer[9].attention.self.key.weight[_start:_end, :]
           # text_value_weight=text_model.encoder.layer[10].attention.self.value.weight[_start:_end, :],text_model.encoder.layer[9].attention.self.value.weight[_start:_end, :]

            text_query_bias=text_model.encoder.layer[image_layerno_1].attention.attn.q.bias,text_model.encoder.layer[image_layerno_2].attention.attn.q.bias 
            text_key_bias=text_model.encoder.layer[image_layerno_1].attention.attn.k.bias,text_model.encoder.layer[image_layerno_2].attention.attn.k.bias
            text_value_bias=text_model.encoder.layer[image_layerno_1].attention.attn.v.bias,text_model.encoder.layer[image_layerno_2].attention.attn.v.bias
            _start = 0
            _end = config["d_model"]
            text_query_weight=text_model.encoder.layer[image_layerno_1].attention.attn.q.weight[_start:_end, :],text_model.encoder.layer[image_layerno_2].attention.attn.q.weight[_start:_end, :]
            text_key_weight=text_model.encoder.layer[image_layerno_1].attention.attn.k.weight[_start:_end, :],text_model.encoder.layer[image_layerno_2].attention.attn.k.weight[_start:_end, :]
            text_value_weight=text_model.encoder.layer[image_layerno_1].attention.attn.v.weight[_start:_end, :],text_model.encoder.layer[image_layerno_2].attention.attn.v.weight[_start:_end, :]




            #image model parameters 
            image_query_bias=image_model.vision_model.encoder.layers[image_layerno_1].self_attn.q_proj.bias,image_model.vision_model.encoder.layers[image_layerno_2].self_attn.q_proj.bias
            image_key_bias=image_model.vision_model.encoder.layers[image_layerno_1].self_attn.k_proj.bias,image_model.vision_model.encoder.layers[image_layerno_2].self_attn.k_proj.bias
            image_value_bias=image_model.vision_model.encoder.layers[image_layerno_1].self_attn.v_proj.bias,image_model.vision_model.encoder.layers[image_layerno_2].self_attn.v_proj.bias

            image_query_weight=image_model.vision_model.encoder.layers[image_layerno_1].self_attn.q_proj.weight[_start:_end, :],image_model.vision_model.encoder.layers[image_layerno_2].self_attn.q_proj.weight[_start:_end, :]
            image_key_weight=image_model.vision_model.encoder.layers[image_layerno_1].self_attn.k_proj.weight[_start:_end, :],image_model.vision_model.encoder.layers[image_layerno_2].self_attn.k_proj.weight[_start:_end, :]
            image_value_weight=image_model.vision_model.encoder.layers[image_layerno_1].self_attn.v_proj.weight[_start:_end, :],image_model.vision_model.encoder.layers[image_layerno_2].self_attn.v_proj.weight[_start:_end, :]
            
            #proj_output=model(config['d_model'], config['ffn_hidden'], config['num_heads'], config['drop_prob'], config['num_layers'], config['kn_vocab_size'], model_audio, image_model, outputs_audio, image_outputs)
            proj_output=model(batch_size_new,text_query_bias,text_key_bias,text_value_bias,text_query_weight,text_key_weight,text_value_weight,image_query_bias,image_key_bias,image_value_bias, image_query_weight,image_key_weight,image_value_weight, 
                              text_layear_output, image_layear_output,image_pooled_output).to(device)
           # out_image = Transformer(audio_query_bias,audio_key_bias,audio_value_bias,audio_query_weight,audio_key_weight,audio_value_weight, image_model, audio_layear_output, image_layear_output)
            text_pooled_output=text_pooled_output/text_pooled_output.norm(dim=-1, keepdim=True)

      
            #image_pooled_output=torch.squeeze(image_pooled_output,dim=0).to(device)
  
            proj_output=proj_output/proj_output.norm(dim=-1, keepdim=True)
            image_last_hidden_state=image_last_hidden_state/image_last_hidden_state.norm(dim=-1, keepdim=True)
            image_pooled_output=image_pooled_output/image_pooled_output.norm(dim=-1, keepdim=True)

            loss = loss_fn(proj_output.view(-1,768), text_pooled_output .view(-1,768))   # compare with pooled_output image to audio

            


            batch_iterator.set_postfix({"loss": f"{loss.item():6.3f}"})

            # Log the loss
            writer.add_scalar('train loss', loss.item(), global_step)
            writer.flush()

            # Backpropagate the loss
            loss.backward()

            # Update the weights
            optimizer.step()
            optimizer.zero_grad(set_to_none=True)

        
            global_step += 1
            


       # print("Numer of errors ",clipvalidation2,clipvalidation ) 
        # Run validation at the end of every epoch
        # Accuracy = run_validation(model, val_dataloader, device, lambda msg: batch_iterator.write(msg), global_step, writer)
        #Accuracy = run_validation(model, val_dataset_both, device,  lambda msg: batch_iterator.write(msg), 0, None, num_examples=100)
        # Save the model at the end of every epoch
        model_filename = get_weights_file_path(config, f"{epoch:02d}")
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'global_step': global_step
        }, model_filename)

if __name__ == '__main__':
    warnings.filterwarnings("ignore")
    config = get_config()
    train_model(config)


Using device: cuda


Found cached dataset parquet (/home/maithri/.cache/huggingface/datasets/detection-datasets___parquet/detection-datasets--coco-64ef6d5414f6b8df/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7)
Found cached dataset parquet (/home/maithri/.cache/huggingface/datasets/detection-datasets___parquet/detection-datasets--coco-64ef6d5414f6b8df/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7)


Size of the dataset : 50000


Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration. Please open a PR/issue to update `preprocessor_config.json` to use `image_processor_type` instead of `feature_extractor_type`. This warning will be removed in v4.40.


image layer number 10 9
No model to preload, starting from scratch
Layers  1


Processing Epoch 00: 14it [01:27,  1.05s/it, loss=-0.757]

In [12]:
#import config
# config the database  
#def get_ds_for_validation():
    

   # dataset_val = load_dataset("csv", data_files="test.csv",split="train")



   # train_dataloader = DataLoader(train_dataset_both, batch_size=config["batch_size"], shuffle=True)
  #  val_dataloader = DataLoader(dataset_val, batch_size=1, shuffle=True)
   # val_dataloader = DataLoader(dataset_val, batch_size=1 )


   # return val_dataloader

In [13]:

#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#print("Using device:", device)
#config = get_config()
#train_dataloader, val_dataloader, tokenizer_src, tokenizer_tgt = get_ds(config)
##train_dataset_both ,val_dataset_both ,train_dataloader, val_dataloader= get_ds()
#val_dataloader= get_ds_for_validation()
#model = get_model(config, tokenizer_src.get_vocab_size(), tokenizer_tgt.get_vocab_size()).to(device)
##model = get_model().to(device)

# Load the pretrained weights
#model_filename = latest_weights_file_path(config)
#model_filename ="weights/tmodel_00.pt"
#print(model_filename)
#state = torch.load(model_filename)
#model.load_state_dict(state['model_state_dict'])



In [14]:
#run_validation(model, val_dataset_both , device,  lambda msg: print(msg), 0, None, num_examples=2500)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)
config = get_config()
train_dataset_both ,val_dataset_both ,train_dataloader, val_dataloader= get_ds()
val_dataset_both2 =val_dataset_both.shard(num_shards=50, index=0)
model = get_model().to(device)
Accuracy = []
for number in range(0, 40, 1):
    if number < 10: 
        str(number)
        number= "0"+ str(number)
    else:
        number=str(number)
    cwd2 = "tmodel_" + number + ".pt"
    cwd = "weights"
    os.path.join(cwd,cwd2)



    # Load the pretrained weights
    model_filename = latest_weights_file_path(config)
    model_filename =os.path.join(cwd,cwd2)
    print(model_filename)
    state = torch.load(model_filename)
    model.load_state_dict(state['model_state_dict'])
    tAccuracy = run_validation(model, val_dataset_both2, device,  lambda msg: print(msg), 0, None, num_examples=2500)
    Accuracy.append(tAccuracy)
    print("mairgh",tAccuracy)
    with open('output.txt', 'a') as testwritefile:
        testwritefile.write(str(number))
        testwritefile.write("layer accuracy ")
        testwritefile.write(str(tAccuracy ))
        testwritefile.write("\n ")
print(Accuracy)
max_value_index = np.argmax(Accuracy)
Maximum_Accuracy = Accuracy[max_value_index]
with open('output.txt', 'a') as testwritefile:
        testwritefile.write("Maximum_Accuracy:")
        testwritefile.write(str(Maximum_Accuracy))
        testwritefile.write("Maximum Accuracy epoch:")
        testwritefile.write(str(max_value_index))
print("Maximum_Accuracy:", Maximum_Accuracy ,"Maximum Accuracy epoch:",max_value_index )